In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Concatenate, LSTM, GRU
from tensorflow.keras.layers import Bidirectional, Multiply

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
train_ = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
print(f"train_: {train_.shape}")
train_.head()

In [ ]:
train_df = pd.read_csv('/kaggle/input/ventilator-transformed-datasets/train_0.csv')
print(f"train_df: {train_df.shape}")
train_df.head()

In [ ]:
targets = train_[['pressure']].to_numpy().reshape(-1, 80)

In [ ]:
del train_
gc.collect()
gc.collect()

In [ ]:
train = train_df.values
del train_df
gc.collect()
gc.collect()

In [ ]:
train = train.reshape(-1, 80, train.shape[-1])

print(f"train: {train.shape} \ntargets: {targets.shape}")

In [ ]:
pressure = targets.squeeze().reshape(-1,1).astype('float32')

P_MIN = np.min(pressure)
P_MAX = np.max(pressure)
P_STEP = (pressure[1] - pressure[0])[0]
print('Min pressure: {}'.format(P_MIN))
print('Max pressure: {}'.format(P_MAX))
print('Pressure step: {}'.format(P_STEP))
print('Unique values:  {}'.format(np.unique(pressure).shape[0]))

del pressure
gc.collect()

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    BATCH_SIZE = tpu_strategy.num_replicas_in_sync * 64
    print("Running on TPU:", tpu.master())
    print(f"Batch Size: {BATCH_SIZE}")
    
except ValueError:
    strategy = tf.distribute.get_strategy()
    BATCH_SIZE = 512
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    print(f"Batch Size: {BATCH_SIZE}")

In [ ]:
BATCH_SIZE = 512

In [ ]:
def dnn_model():
    
    x_input = Input(shape=(train.shape[-2:]))
    
    x1 = Bidirectional(LSTM(units=768, return_sequences=True))(x_input)
    x2 = Bidirectional(LSTM(units=512, return_sequences=True))(x1)
    x3 = Bidirectional(LSTM(units=384, return_sequences=True))(x2)
    x4 = Bidirectional(LSTM(units=256, return_sequences=True))(x3)
    x5 = Bidirectional(LSTM(units=128, return_sequences=True))(x4)
    
    z2 = Bidirectional(GRU(units=384, return_sequences=True))(x2)
    
    z31 = Multiply()([x3, z2])
    z31 = BatchNormalization()(z31)
    z3 = Bidirectional(GRU(units=256, return_sequences=True))(z31)
    
    z41 = Multiply()([x4, z3])
    z41 = BatchNormalization()(z41)
    z4 = Bidirectional(GRU(units=128, return_sequences=True))(z41)
    
    z51 = Multiply()([x5, z4])
    z51 = BatchNormalization()(z51)
    z5 = Bidirectional(GRU(units=64, return_sequences=True))(z51)
    
    x = Concatenate(axis=2)([x5, z2, z3, z4, z5])
    
    x = Dense(units=128, activation='selu')(x)
    
    x_output = Dense(units=1)(x)

    model = Model(inputs=x_input, outputs=x_output, 
                  name='DNN_Model')
    return model

In [ ]:
model = dnn_model()
model.summary()

In [ ]:
with tpu_strategy.scope():
    
    VERBOSE = 1
    test_preds = []
    kf = KFold(n_splits=7, shuffle=True, random_state=2021)
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        if fold == 0:
            X_train, X_valid = train[train_idx], train[test_idx]
            y_train, y_valid = targets[train_idx], targets[test_idx]

            model = dnn_model()
            model.compile(optimizer="adam", loss="mae")

            model.fit(X_train, y_train, 
                      validation_data=(X_valid, y_valid), 
                      epochs=3,
                      verbose=VERBOSE,
                      batch_size=BATCH_SIZE)


In [ ]:
BATCH_SIZE = 1024

with tpu_strategy.scope():
    
    VERBOSE = 1
    test_preds = []
    kf = KFold(n_splits=7, shuffle=True, random_state=2021)
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        if fold == 0:
            X_train, X_valid = train[train_idx], train[test_idx]
            y_train, y_valid = targets[train_idx], targets[test_idx]

            model = dnn_model()
            model.compile(optimizer="adam", loss="mae")


            model.fit(X_train, y_train, 
                      validation_data=(X_valid, y_valid), 
                      epochs=3,
                      verbose=VERBOSE,
                      batch_size=BATCH_SIZE)

In [ ]:
BATCH_SIZE = 2048

with tpu_strategy.scope():
    
    VERBOSE = 1
    test_preds = []
    kf = KFold(n_splits=7, shuffle=True, random_state=2021)
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        if fold == 0:
            X_train, X_valid = train[train_idx], train[test_idx]
            y_train, y_valid = targets[train_idx], targets[test_idx]

            model = dnn_model()
            model.compile(optimizer="adam", loss="mae")


            model.fit(X_train, y_train, 
                      validation_data=(X_valid, y_valid), 
                      epochs=3,
                      verbose=VERBOSE,
                      batch_size=BATCH_SIZE)

In [ ]:
BATCH_SIZE = 4096

with tpu_strategy.scope():
    
    VERBOSE = 1
    test_preds = []
    kf = KFold(n_splits=7, shuffle=True, random_state=2021)
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        if fold == 0:
            X_train, X_valid = train[train_idx], train[test_idx]
            y_train, y_valid = targets[train_idx], targets[test_idx]

            model = dnn_model()
            model.compile(optimizer="adam", loss="mae")

            model.fit(X_train, y_train, 
                      validation_data=(X_valid, y_valid), 
                      epochs=3,
                      verbose=VERBOSE,
                      batch_size=BATCH_SIZE)

In [ ]:
BATCH_SIZE = 5000

with tpu_strategy.scope():
    
    VERBOSE = 1
    test_preds = []
    kf = KFold(n_splits=7, shuffle=True, random_state=2021)
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        if fold == 0:
            X_train, X_valid = train[train_idx], train[test_idx]
            y_train, y_valid = targets[train_idx], targets[test_idx]

            model = dnn_model()
            model.compile(optimizer="adam", loss="mae")

        
            model.fit(X_train, y_train, 
                      validation_data=(X_valid, y_valid), 
                      epochs=3,
                      verbose=VERBOSE,
                      batch_size=BATCH_SIZE)

In [ ]:
BATCH_SIZE = 6000

with tpu_strategy.scope():
    
    VERBOSE = 1
    test_preds = []
    kf = KFold(n_splits=7, shuffle=True, random_state=2021)
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        if fold == 0:
            X_train, X_valid = train[train_idx], train[test_idx]
            y_train, y_valid = targets[train_idx], targets[test_idx]

            model = dnn_model()
            model.compile(optimizer="adam", loss="mae")


            model.fit(X_train, y_train, 
                      validation_data=(X_valid, y_valid), 
                      epochs=3,
                      verbose=VERBOSE,
                      batch_size=BATCH_SIZE)

In [ ]:
BATCH_SIZE = 6500

with tpu_strategy.scope():
    
    VERBOSE = 1
    test_preds = []
    kf = KFold(n_splits=7, shuffle=True, random_state=2021)
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        if fold == 0:
            X_train, X_valid = train[train_idx], train[test_idx]
            y_train, y_valid = targets[train_idx], targets[test_idx]

            model = dnn_model()
            model.compile(optimizer="adam", loss="mae")


            model.fit(X_train, y_train, 
                      validation_data=(X_valid, y_valid), 
                      epochs=3,
                      verbose=VERBOSE,
                      batch_size=BATCH_SIZE)

In [ ]:
BATCH_SIZE = 6800

with tpu_strategy.scope():
    
    VERBOSE = 1
    test_preds = []
    kf = KFold(n_splits=7, shuffle=True, random_state=2021)
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        if fold == 0:
            X_train, X_valid = train[train_idx], train[test_idx]
            y_train, y_valid = targets[train_idx], targets[test_idx]

            model = dnn_model()
            model.compile(optimizer="adam", loss="mae")


            model.fit(X_train, y_train, 
                      validation_data=(X_valid, y_valid), 
                      epochs=3,
                      verbose=VERBOSE,
                      batch_size=BATCH_SIZE)

In [ ]:
BATCH_SIZE = 7000

with tpu_strategy.scope():
    
    VERBOSE = 1
    test_preds = []
    kf = KFold(n_splits=7, shuffle=True, random_state=2021)
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        if fold == 0:
            X_train, X_valid = train[train_idx], train[test_idx]
            y_train, y_valid = targets[train_idx], targets[test_idx]

            model = dnn_model()
            model.compile(optimizer="adam", loss="mae")


            model.fit(X_train, y_train, 
                      validation_data=(X_valid, y_valid), 
                      epochs=3,
                      verbose=VERBOSE,
                      batch_size=BATCH_SIZE)

In [ ]:
BATCH_SIZE = 7100

with tpu_strategy.scope():
    
    VERBOSE = 1
    test_preds = []
    kf = KFold(n_splits=7, shuffle=True, random_state=2021)
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        if fold == 0:
            X_train, X_valid = train[train_idx], train[test_idx]
            y_train, y_valid = targets[train_idx], targets[test_idx]

            model = dnn_model()
            model.compile(optimizer="adam", loss="mae")


            model.fit(X_train, y_train, 
                      validation_data=(X_valid, y_valid), 
                      epochs=3,
                      verbose=VERBOSE,
                      batch_size=BATCH_SIZE)